In [1]:
import sys
import pandas as pd
import numpy as np
import requests
from time import sleep
sys.path.append('/My Drive/Excels útiles/')
from functions import *
from tqdm.notebook import tqdm
import math
from concurrent.futures import ThreadPoolExecutor
import itertools
import json
import random

# Strategy:

Goal: Have most of the meli_item_id from MELI_API, taking into account that we need to prioritize most import sellers somehow.-

Strategy 1:
1. get all meli_category_ids
2. scrape top +1.000 SKU for each meli_category_id (sorted by "most importants")
3. get all meli_seller_ids related to step 2)
4. scrape +2.000 SKUS from each meli_seller_ids related to 3) (sorted by "most importants", "cheaper" and "expensive"), remember to use scrape_meli_skus_largo()

## 1) get all meli_categorys_ids

In [ ]:
#categories_df = pd.DataFrame(scrape_category_id_without_childrens('MLC'))
#categories_df

In [ ]:
#categories_df.to_csv("meli_categories.csv")

In [2]:
path_categories = 'G:\My Drive\Proyectos\[CL Scrape] Full Meli Scrape\\20220901\meli_categories.csv'
categories = pd.read_csv(
    path_categories, usecols = ['id', 'name', 'total_items_in_this_category']).rename(
        columns={'total_items_in_this_category':'total_items'}).sort_values(
            by = 'total_items', ascending=False)
categories = categories[categories['total_items'] > 0] # ~900 root_categories doesn't have any skus
categories = categories.reset_index().drop(columns = 'index')
categories['ranges'] =  np.where(categories.total_items <=2000, "<=2k",
                        np.where(categories.total_items <=5000,  "2.0k<x<5.0k",
                        np.where(categories.total_items <=10000, "5.0k<x<10.0k",
                        np.where(categories.total_items <=20000, "10.0k<x<20.0k",
                        np.where(categories.total_items <=50000, "20.0k<x<50.0k",
                        np.where(categories.total_items <=125000, "50.0k<x<125.0k",
                        np.where(categories.total_items <=300000, "125.0k<x<300.0k",
                        np.where(categories.total_items <=750000, "300.0k<x<750.0k",
                        "750k>x")
                        )))))))
categories['percx100'] = categories['total_items']*100/np.sum(categories['total_items'])
categories['url'] = categories['id'].apply(lambda id: f"https://api.mercadolibre.com/sites/MLC/search?category={id}")
categories

,id,name,total_items,ranges,percx100,url
0,MLC1196,Libros Físicos,3523063,750k>x,11.815423,https://api.mercadolibre.com/sites/MLC/search?...
1,MLC59887,Cubre Asientos,627799,300.0k<x<750.0k,2.105472,https://api.mercadolibre.com/sites/MLC/search?...
2,MLC157694,Carcasas y Fundas,423198,300.0k<x<750.0k,1.419294,https://api.mercadolibre.com/sites/MLC/search?...
3,MLC158425,Vestidos,373022,300.0k<x<750.0k,1.251017,https://api.mercadolibre.com/sites/MLC/search?...
4,MLC22887,Pastillas de Freno,325381,300.0k<x<750.0k,1.091242,https://api.mercadolibre.com/sites/MLC/search?...
...,...,...,...,...,...,...
7431,MLC440600,Carros Eléctricos,1,<=2k,0.000003,https://api.mercadolibre.com/sites/MLC/search?...
7432,MLC432113,Portaflechas,1,<=2k,0.000003,https://api.mercadolibre.com/sites/MLC/search?...
7433,MLC437005,Veleros,1,<=2k,0.000003,https://api.mercadolibre.com/sites/MLC/search?...
7434,MLC437022,Accesorios,1,<=2k,0.000003,https://api.mercadolibre.com/sites/MLC/search?...


In [3]:
categories.total_items.sum()

29817494

In [ ]:
data_agrupada = pd.DataFrame(categories.groupby(categories.ranges).agg(
    total_items=pd.NamedAgg(column="total_items", aggfunc="sum"),
    total_categories  =pd.NamedAgg(column="id", aggfunc="count")
    )
    )
data_agrupada['perc_items'] = np.round(data_agrupada['total_items']*100/data_agrupada['total_items'].sum(), 2)
data_agrupada['orden'] = [4,7,2,5,8,3,6,9,1]
data_agrupada = data_agrupada.sort_values(by = 'orden')
data_agrupada['perc_categories'] = np.round(data_agrupada['total_categories']*100/data_agrupada['total_categories'].sum(), 2)
data_agrupada['suma_acum_items'] = np.round(np.cumsum(data_agrupada['perc_items']),1)
data_agrupada['suma_acum_categorías'] = np.round(np.cumsum(data_agrupada['perc_categories']),2)
data_agrupada[['total_items', 'perc_items','suma_acum_items','total_categories', 'perc_categories', 'suma_acum_categorías', 'orden']]
#plt.barh(x=categories['ranges'], , orientation='horizontal')
#categories.ranges.value_counts('%')

No hay un buen análisis de aquí, porque en realidad necesitamos saber donde hay más $$ para saber si scrapeando el 80/20 podemos hacer un buen scrapeo.

## 2) scrape top +1.000 SKU for each meli_category_id
### sorted by "most importants" (at least)

### Stragegy_2: ¿Cómo obtener todas las particiones para una categoría tal que cada partición sea de 1.000 SKUS o menos?

1. Crear una función recursiva que si la partición actual tiene más de 1.000 SKUS, entonces necesita hacer una partición
2. ¿cuantos particiones se harán? p veces, donde si p es la cantidad de particiones y s es la cantidad de skus de la partición actual
    entonces p = int(s/1000)+1
3. Teniendo la cantidad de particiones, se harán tantas nuevas consultas como nuevas particiones, y en cada nueva partición se hará nuevamente la pregunta ¿Tiene más de 2k de SKUS esta partición?
4. Así sucesivamente hasta que la partición que se consulte tenga 1.000 o menos skus y esa se agrega a un listado de particiones "válidas".
5. La idea es que el algoritmo también pare si es que la diferencia entre la parte altas y baja del rango es 500 pesos o menos (las particiones serán por rangos de precios: Por ejemplo, los productos de una categoría donde los precios estén entre $5.000 y $10.000), entonces no siga particionando y se agregue la actual consulta a la listado de "válidas".

### También necesitamos crear 3 funciones

1. función que entregue todas las URLs a scrapear con .get(url) que tengan 1.000 SKUS o menos, esta será la más simple
2. función que entregue todas las URLs a scrapear con .get(url) que tengan más de 1.000 SKUS pero no más de 2.000 SKUS  (esto para buscar por &sort=price_asc' y '&sort=price_desc')
3. función que desarolle Strategy_2

### Funciones

In [2]:
def add_value_to_a_list_and_return_it(lista: list, new_value: any, lugar: int)-> list:
    if lugar != len(lista):
        lista[lugar] = new_value
    else:
        lista.insert(lugar, new_value)
    return lista

def crea_rangos(lista: list) -> list:
    results = []
    for i in range(len(lista)):
        try:
            new_range = str(lista[i])+'-'+str(lista[i+1])
            results.insert(len(results), new_range)
        except:
            pass  
    return results

def partition_by_price(df_prices: pd.DataFrame) -> pd.DataFrame:
    df_prices['n_partitions'] = df_prices.apply(lambda row: 1 if (row['results'] <= 1000) else 2, axis = 1)
    df_prices['min'] = df_prices['id'].apply(lambda row: 0 if (row[:1]  == '*') else float(row[:row.find("-")-2  ])*(1 if (row[:row.find("-")].find('E') == -1)   else pow(10,float(row[ row[:row.find("-")].find('E')+1 : row.find("-")]))))
    df_prices['max'] = df_prices['id'].apply(lambda row: 0 if (row[-1:] == '*') else float(row[row.find("-")+1:-2])*(1 if (row[row.find("-")+1:].find('E') == -1) else pow(10,float(row[ row.find("-")+row[row.find("-"):].find('E')+1 :]))))
    df_prices['limit_partitions'] = df_prices.apply(lambda row: list(np.linspace(start=row['min'], stop=row['max'] if (row['max'] != 0) else row['min']*1.5, num=row['n_partitions']+1 if (row['max'] != 0) else row['n_partitions'], dtype=int,  endpoint=True)) ,axis= 1)
    df_prices['limit_partitions'] = df_prices.apply(lambda row: add_value_to_a_list_and_return_it(row['limit_partitions'], '*', 0) if (row['id'][:1] == '*') else row['limit_partitions'], axis=1)
    df_prices['limit_partitions'] = df_prices.apply(lambda row: add_value_to_a_list_and_return_it(row['limit_partitions'], '*', len(row['limit_partitions'])) if (row['id'][-1:] == '*') else row['limit_partitions'], axis=1)
    df_prices['add_for_close_diff'] = df_prices.apply(lambda row: np.abs(row['max'] - row['min']) <= 50, axis=1)
    df_prices['partitions'] = df_prices['limit_partitions'].apply(lambda row: crea_rangos(row))
    
    return df_prices

def reintentos_request(url:str) -> str:
    segundos = 1
    try:
        resultado = requests.get(url)
    except:
        print(f"falló un request({segundos}). {url}")
        sleep(segundos)
        code = 400
        while code != 200:
            try:
                resultado = requests.get(url)
                code = resultado.status_code
            except:
                segundos += 1
                sleep(segundos)
                print(f"falló un request({segundos}). {url}")
                pass
    return resultado

def scrape_meli_urls_from_category_id_2k_or_more(url_a_particionar: str)-> list:
    urls_validas = {}

    check_inicial = reintentos_request(url_a_particionar).json()
    #print(f"estoy haciendo la url = {url_a_particionar}. la cual tiene {check_inicial['paging']['total']} SKUS")

    if check_inicial['paging']['total'] > 1000:
        try:
            #dado que ambos lados del rango de precio son inclusivos, entonces se duplica un poco la información, se podría acortar el time-consuming, pero guardando en dict se solucionan los duplicados.
            temp_prices = pd.DataFrame(pd.DataFrame(check_inicial['available_filters']).set_index('id').filter(items=['price'], axis=0)['values'][0])
            
        except:
            min = float(url_a_particionar[url_a_particionar.find('&price=')+7:url_a_particionar.find('-')] if (url_a_particionar[url_a_particionar.find('&price=')+7:url_a_particionar.find('-')] != '*') else 0)
            max = float(url_a_particionar[url_a_particionar.find('-')+1:] if (url_a_particionar[url_a_particionar.find('-')+1:] != '*') else 1.5*min)
            middle_left = float(math.trunc((int(min) + int(max))/2))
            middle_right = float(math.trunc((int(min) + int(max))/2))+1

            temp_prices = pd.DataFrame({'id':[f"{min}-{middle_left}", f"{middle_right}-{max}" ], 'results': [
                reintentos_request(f"{url_a_particionar[:url_a_particionar.find('&price=')+7]}{f'{min}-{middle_left}'}").json()['paging']['total'],
                reintentos_request(f"{url_a_particionar[:url_a_particionar.find('&price=')+7]}{f'{middle_right}-{max}'}").json()['paging']['total']
            ]})
        #print(temp_prices)
        temp_prices = partition_by_price(temp_prices)

        temp_prices_a_agregar     = temp_prices[(temp_prices['n_partitions'] == 1) & (temp_prices['results'] > 0)  | ( temp_prices['add_for_close_diff'])]
        temp_prices_a_particionar = temp_prices[(temp_prices['n_partitions'] > 1 ) & (temp_prices['results'] > 0)  &  (~temp_prices['add_for_close_diff'])]

        urls_a_agregar_a_validas =  {}
        for index, row in temp_prices_a_agregar.iterrows():
            temp_url = f"{url_a_particionar[:len(url_a_particionar) if url_a_particionar.find('&') == -1 else url_a_particionar.find('&')]}&price={row['partitions'][0]}"
            urls_a_agregar_a_validas[temp_url] = row['results']
        urls_validas.update(urls_a_agregar_a_validas)
        
        temp_prices_a_particionar = [f"{url_a_particionar[:len(url_a_particionar) if url_a_particionar.find('&') == -1 else url_a_particionar.find('&')]}&price={row[element]}" for row in temp_prices_a_particionar['partitions'] for element in range(len(row))]

        #print(urls_validas)
        #print(urls_a_agregar_a_validas)
        #print(temp_prices_a_particionar)

        if len(temp_prices_a_particionar)>0:
            for i in temp_prices_a_particionar:
                urls_validas.update(scrape_meli_urls_from_category_id_2k_or_more(i))
    else:
        #print(f"Esta tiene menos de 1.000, tiene {check_inicial['paging']['total']} así que la agregué")
        if check_inicial['paging']['total'] >0:
            urls_validas[url_a_particionar] = check_inicial['paging']['total']
    return urls_validas

def scrape_meli_skus_from_dict_urls(urls_dict_url_plus_total_items: dict) -> pd.DataFrame:
    results = pd.DataFrame({}, columns=[
        'meli_item_id',
        'meli_seller_id',
        'item_name',
        'original_price',
        'price',
        'condition',
        'buying_mode',
        'shipping',
        'official_store_id',
        'category_id',
        'sold_quantity'
    ])
    for key, total_items in urls_dict_url_plus_total_items.items():
        try:
            cantidad_de_items = reintentos_request(key).json()['paging']['total']
        except:
            cantidad_de_items = total_items
        multiplos = list(range(0,cantidad_de_items,50))[0:21]
        available_sorts = ['','&sort=price_asc','&sort=price_desc']
        #Caso en que total_items (largo de la categoría) NO es > 1050
        if cantidad_de_items <= 1050:
            for j in multiplos:
                #print(f'{key}&offset={j}')
                req = reintentos_request(url=f'{key}&offset={j}').json()
                try:
                    resultados = req['results']
                    for i in resultados:
                        
                        try:
                            original_price = str(int(i['original_price']))
                        except:
                            original_price = np.nan

                        try:
                            price = str(int(i['price']))
                        except:
                            price = np.nan
                        
                        try:
                            official_store_id = str(int(i['official_store_id']))
                        except:
                            official_store_id = np.nan

                        new_row  = pd.DataFrame({
                            'meli_item_id' : i['id'],
                            'meli_seller_id':i['seller']['id'],
                            'item_name': i['title'],
                            'original_price': original_price,
                            'price': price,
                            'condition': i['condition'],
                            'buying_mode' : i['buying_mode'],
                            'shipping' : str(json.dumps(i['shipping'])),
                            'official_store_id': official_store_id,
                            'category_id': i['category_id'],
                            'sold_quantity': str(int(i['sold_quantity']))
                        },
                        columns=[
                            'meli_item_id',
                            'meli_seller_id',
                            'item_name',
                            'original_price',
                            'price',
                            'condition',
                            'buying_mode',
                            'shipping',
                            'official_store_id',
                            'category_id',
                            'sold_quantity'
                            ], index=[0])
                        results = pd.concat([results, new_row], ignore_index=True)
                except:
                    #print(f'{key}&offset={j}')
                    pass

        #Caso en que la cantidad de la categoría tiene más de 1050 skus
        else:
            for sort_type in available_sorts:
                for j in multiplos:
                    req = reintentos_request(url=f'{key}&offset={j}{sort_type}').json()
                    try:
                        for i in req['results']:                    
                            #print(f'{key}&offset={j}{sort_type}')    
                            try:
                                original_price = str(int(i['original_price']))
                            except:
                                original_price = np.nan

                            try:
                                price = str(int(i['price']))
                            except:
                                price = np.nan
                            
                            try:
                                official_store_id = str(int(i['official_store_id']))
                            except:
                                official_store_id = np.nan

                            new_row  = pd.DataFrame({
                                'meli_item_id' : i['id'],
                                'meli_seller_id':i['seller']['id'],
                                'item_name': i['title'],
                                'original_price': original_price,
                                'price': price,
                                'condition': i['condition'],
                                'buying_mode' : i['buying_mode'],
                                'shipping' : str(json.dumps(i['shipping'])),
                                'official_store_id': official_store_id,
                                'category_id': i['category_id'],
                                'sold_quantity': str(int(i['sold_quantity']))
                            },
                            columns=[
                                'meli_item_id',
                                'meli_seller_id',
                                'item_name',
                                'original_price',
                                'price',
                                'condition',
                                'buying_mode',
                                'shipping',
                                'official_store_id',
                                'category_id',
                                'sold_quantity'
                                ], index=[0])
                            results = pd.concat([results, new_row], ignore_index=True)
                    except:
                        #print(f'{key}&offset={j}')
                        pass
    return results.drop_duplicates()
    
def get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice(listWithUrlWithoutPrice:list) -> dict:
    resultsdict = {}
    for urlWithoutPrice in listWithUrlWithoutPrice:
        resultsdict.update(scrape_meli_urls_from_category_id_2k_or_more(urlWithoutPrice))
    return resultsdict

## 3) get all meli_seller_ids related to step 2)

#### 4) Time to big scrape!

(scrape all urls from all root_categories)

ALL URLS

In [ ]:
categories_url = categories['url']
categories_url

Si la URL es de 15K en adelante, debe hacerse por separado, el resto, en batches de 1.000

In [ ]:
categories

In [ ]:
pareado = pd.DataFrame({}, columns=['cluster', 'id'])

for i in range(len(categories)//2):
    new_row = pd.DataFrame({
        'cluster': i,
        'id': [categories['id'][i], categories['id'][len(categories)-1-i]],
    }, columns=pareado.columns
        )
    #print(new_row)
    pareado = pd.concat([pareado, new_row], ignore_index=True)

pareado

In [ ]:
categories = categories.merge(pareado, on =['id'])
categories

In [ ]:
divided_categories_url = categories.groupby(['cluster'])['url'].apply(list).reset_index()['url']
divided_categories_url

In [ ]:
divided_categories_url[0] #malditos libros

In [ ]:
all_urls_result = {}

In [ ]:
div0 = [
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=*-10000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=10000.0-20000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=20000.0-30000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=30000.0-40000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=40000.0-50000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=50000.0-60000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=60000.0-70000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=70000.0-80000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=80000.0-90000.0'  ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=90000.0-100000.0' ],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=100000.0-125000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=125000.0-150000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=150000.0-175000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=175000.0-200000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=200000.0-225000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=225000.0-250000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=250000.0-275000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=275000.0-300000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=300000.0-325000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=325000.0-350000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=350000.0-375000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=375000.0-400000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=400000.0-500000.0'],
    ['https://api.mercadolibre.com/sites/MLC/search?category=MLC1196&price=500000.0-*']
    ] #libros"
div1 = divided_categories_url[1:800]
div2 = divided_categories_url[800:1600]
div3 = divided_categories_url[1600:2400]
div4 = divided_categories_url[2400:3200]
div5 = divided_categories_url[3200:]

current = div0 # Hay que ir cambiando este valor por cada divX
with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice, current))
for x in results:
    all_urls_result.update(x)
len(all_urls_result) 

In [ ]:
current = div1 # Hay que ir cambiando este valor por cada divX
with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice, current))
for x in results:
    all_urls_result.update(x)
len(all_urls_result) 

In [ ]:
current = div2 # Hay que ir cambiando este valor por cada divX
with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice, current))
for x in results:
    all_urls_result.update(x)
len(all_urls_result) 

In [ ]:
current = div3 # Hay que ir cambiando este valor por cada divX
with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice, current))
for x in results:
    all_urls_result.update(x)
len(all_urls_result) 

In [ ]:
current = div4 # Hay que ir cambiando este valor por cada divX
with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice, current))
for x in results:
    all_urls_result.update(x)
len(all_urls_result)

In [ ]:
current = div5 # Hay que ir cambiando este valor por cada divX
with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(get_dict_withUrlPlusPriceAndTotaItems_from_listWithUrlWithoutPrice, current))
for x in results:
    all_urls_result.update(x)
len(all_urls_result) 

In [ ]:
len(all_urls_result) # 1) scrape=40258 | 2) scrape=40376 | 3) scrape=43553

In [ ]:
pd.DataFrame(all_urls_result, index=["total_items"]).T.to_csv("current_all_urls_result.csv")

In [3]:
all_urls_result_csv = pd.read_csv("G:\My Drive\Proyectos\[CL Scrape] Full Meli Scrape\\20220901\current_all_urls_result.csv").rename(columns={'Unnamed: 0':'url'})
all_urls_result_csv = all_urls_result_csv[all_urls_result_csv['total_items'] > 0]
all_urls_result = {}

for index, row in all_urls_result_csv.iterrows():
    all_urls_result[str(row['url'])] = row['total_items']
all_urls_result;

## 4) scrape +2.000 SKUS from each meli_seller_ids related to 3)
### sorted by "most importants", "cheaper" and "expensive"
### remember to use scrape_meli_skus_largo() also

In [ ]:
len(all_urls_result)

In [ ]:
all_urls_result;

# PRUEBAS

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones
batch0 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(0   , 1000, 1)]


current = batch0

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch0.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones
batch1 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(1000  , 2000, 1)]

current = batch1

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch1.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch2 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(2000  , 3000, 1)]

current = batch2

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch2.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch3 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(3000 ,  4000, 1)]

current = batch3

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch3.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch4 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(4000 ,  5000, 1)]

current = batch4

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch4.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch5 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(5000 ,  6000, 1)]


current = batch5

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch5.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch6 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(6000 ,  7000, 1)]

current = batch6

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch6.csv")

In [ ]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch7 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(7000 ,  8000, 1)]


current = batch7

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch7.csv")

In [4]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch8 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(8000 ,  9000, 1)]

current = batch8

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch8.csv")

In [5]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch9 =  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(9000 , 10000, 1)]

current = batch9

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch9.csv")

In [6]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch10=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(10000, 11000, 1)]

current = batch10

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch10.csv")

In [7]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch11 = [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(11000, 12000, 1)]

current = batch11

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch11.csv")

In [8]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch12=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(12000, 13000, 1)]

current = batch12

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch12.csv")

In [9]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch13=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(13000, 14000, 1)]

current = batch13

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch13.csv")

In [10]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch14=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(14000, 15000, 1)]

current = batch14

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch14.csv")

In [11]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch15=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(15000, 16000, 1)]

current = batch15

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch15.csv")

In [12]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch16=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(16000, 17000, 1)]

current = batch16

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch16.csv")

In [13]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch17=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(17000, 18000, 1)]

current = batch17

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch17.csv")

In [14]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch18=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(18000, 19000, 1)]

current = batch18

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch18.csv")

In [15]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch19=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(19000, 20000, 1)]

current = batch19

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch19.csv")

In [16]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones
batch20=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(20000, 21000, 1)]

current = batch20

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch20.csv")

falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC161587&price=237501-250000&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC393045&price=25000-27500&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC159295&price=35000-41250&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC159295&price=105000-152500&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC373325&price=30000-40000&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC440279&price=23750-25000&offset=250
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC439870&price=16251-16563&offset=300
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC161567&price=27500-30000&offset=200


In [17]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch21=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(21000, 22000, 1)]

current = batch21

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch21.csv")

In [18]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch22=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(22000, 23000, 1)]

current = batch22

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch22.csv")

In [19]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch23=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(23000, 24000, 1)]

current = batch23

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch23.csv")

In [20]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch24=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(24000, 25000, 1)]

current = batch24

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch24.csv")

In [21]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch25=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(25000, 26000, 1)]

current = batch25

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch25.csv")

In [22]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch26=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(26000, 27000, 1)]


current = batch26

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch26.csv")

In [23]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch27=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(27000, 28000, 1)]

current = batch27

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch27.csv")

In [24]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch28=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(28000, 29000, 1)]

current = batch28

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch28.csv")

In [25]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch29=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(29000, 30000, 1)]

current = batch29

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch29.csv")

In [26]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch30=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(30000, 31000, 1)]

current = batch30

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch30.csv")

In [27]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch31=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(31000, 32000, 1)]

current = batch31

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch31.csv")

In [28]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch32=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(32000, 33000, 1)]

current = batch32

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch32.csv")

In [29]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch33=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(33000, 34000, 1)]

current = batch33

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch33.csv")

In [30]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch34=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(34000, 35000, 1)]

current = batch34

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch34.csv")

In [31]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch35=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(35000, 36000, 1)]

current = batch35

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch35.csv")

In [32]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch36=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(36000, 37000, 1)]

current = batch36

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch36.csv")

In [33]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch37=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(37000, 38000, 1)]

current = batch37

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch37.csv")

In [34]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch38=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(38000, 39000, 1)]

current = batch38

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch38.csv")

In [35]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch39=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(39000, 40000, 1)]

current = batch39

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch39.csv")

In [36]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch40=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(40000, 41000, 1)]

current = batch40

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch40.csv")

falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC161448&offset=650
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC441039&price=95497-95503&offset=600
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC441039&price=109987-109993&offset=600


In [37]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch41=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(41000, 42000, 1)]

current = batch41

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch41.csv")

falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC431656&price=9500-35000&offset=250
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC400500&price=35000-*&offset=200
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC413013&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC416202&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC174494&offset=250
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC455747&offset=200
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC417757&price=*-25000&offset=250
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC32650&offset=150
falló un request(1). https://api.mercadolibre.com/sites/MLC/search?category=MLC437374&price=22500-25000&offset=100
falló un request(1). https://api.mercadolibre.com/sites/MLC/searc

In [38]:
resultados_full_meli_scrape = pd.DataFrame({}, columns=[
    'meli_item_id', 'meli_seller_id', 'item_name', 'original_price', 'price', 'condition', 'buying_mode', 'shipping',
    'official_store_id', 'category_id', 'sold_quantity'])

#particiones

batch42=  [dict(itertools.islice(all_urls_result.items(), i, i+1)) for i in range(42000, 43000, 1)]

current = batch42

with ThreadPoolExecutor(max_workers=len(current)*2) as executor:
    results = list(executor.map(scrape_meli_skus_from_dict_urls, current))
for x in results:
    resultados_full_meli_scrape = pd.concat([resultados_full_meli_scrape,x], ignore_index=True)
len(resultados_full_meli_scrape)
resultados_full_meli_scrape.to_csv("20220901 batch42.csv")